In [ ]:
#do ya imports and shiii

#!pip install ddsp[training]
import ddsp
import tensorflow as tf


ValueError: Arg specs do not match: original=FullArgSpec(args=['input', 'dtype', 'name', 'layout'], varargs=None, varkw=None, defaults=(None, None, None), kwonlyargs=[], kwonlydefaults=None, annotations={}), new=FullArgSpec(args=['input', 'dtype', 'name'], varargs=None, varkw=None, defaults=(None, None), kwonlyargs=[], kwonlydefaults=None, annotations={}), fn=<function ones_like_v2 at 0x16a12ed30>

In [ ]:
from ddsp.training.data_preparation import prepare_tfrecord_lib

input_files = [
    "Sounds/singlecoil/*.wav",
    "Sounds/humbucker/*.wav"
]

prepare_tfrecord_lib.prepare_tfrecord(
    input_audio_filepatterns=input_files,
    output_tfrecord_path="guitar_dataset.tfrecord",
    num_shards=1,
    sample_rate=16000, #SAMPLE_RATE PERAMETER?
    frame_rate=250,
    example_secs=4.0, #EXAMPLE_SECS PERAMETER?
    hop_secs=1.0,
    eval_split_fraction=0.0,
    chunk_secs=20.0
)


# this is how you preprocess the data, lmao, its the actual code from the gitRepo
#heres that: https://github.com/magenta/ddsp/blob/main/ddsp/training/data_preparation/ddsp_prepare_tfrecord.py

#this is the same as using google's library like this in the command line

'''ddsp_prepare_tfrecord \
  --input_audio_filepatterns="/full/path/to/guitar_data/*/*.wav" \
  --output_tfrecord_path="/full/path/to/output/guitar_dataset.tfrecord" \
  --num_shards=1
'''
#idrk what a shard is

'''This contains:

    The waveform

    The extracted pitch

    Loudness

    Other metadata

This file will be your training data.'''

#this is kinda like a train-test-split, but a step before that acually happens
# its a "preprocess data" function. we love google


In [ ]:
#convert the MOTHAFUKIN data from a preprocessed .tfrecord file to a tensorflow guitar_dataset

from ddsp.training.data import TFRecordProvider

provider = TFRecordProvider(
    file_pattern='guitar_dataset.tfrecord',
    example_secs=4, #EXAMPLE_SECS PERAMETER?
    sample_rate=16000 #SAMPLE_RATE PERAMETER?
)

#batches the dataset into sizes 1
dataset = provider.get_dataset(batch_size=1)

#grab first record
example = next(iter(dataset))

#example looks like this
'''{
  'audio': <Tensor shape=(1, 64000)>,  # 4s of audio at 16kHz
  'f0_hz': <Tensor shape=(1, 1000)>,   # 250 Hz frame rate × 4 seconds
  'loudness_db': <Tensor shape=(1, 1000)>,
  ...
}
'''


In [ ]:
#with example, you can do this

import matplotlib.pyplot as plt

plt.plot(example['f0_hz'][0].numpy(), label='Pitch (Hz)')
plt.plot(example['loudness_db'][0].numpy(), label='Loudness (dB)')
plt.legend()
plt.show()


In [ ]:
#Here's the actual model

from ddsp.training.models import Autoencoder
from ddsp.training.encoders import FcEncoder
from ddsp.training.decoders import RnnFcDecoder

model = Autoencoder(
    encoder=FcEncoder(),       # [source](https://github.com/magenta/ddsp/blob/main/ddsp/training/encoders.py#L39)
    decoder=RnnFcDecoder()     # [source](https://github.com/magenta/ddsp/blob/main/ddsp/training/decoders.py#L90)
)


In [ ]:
#Loss, comparing predicted spectrogram to actual spectrogram

from ddsp.training.losses import MultiscaleSpectralLoss
loss_fn = MultiscaleSpectralLoss()


In [ ]:
#Training loop

import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        outputs = model(batch, training=True)
        loss = loss_fn(batch['audio'], outputs['audio'])
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


In [ ]:
# train that bitch

for step, batch in enumerate(dataset.take(1000)):
    loss = train_step(batch)
    if step % 100 == 0:
        print(f"Step {step}: Loss = {loss.numpy():.4f}")


In [ ]:
#Preprocess a new audio clip and then

output = model(example, training=False)
synth_audio = output['audio']



import soundfile as sf
sf.write("output.wav", synth_audio.numpy()[0], 16000)
